In [5]:
import pandas as pd
import numpy as np
from new_user_similarity import find_similar_index
from recommend_data_road import recommend_all
from recommend_data_road import recommendation_system
import warnings
warnings.filterwarnings("ignore")

In [6]:
# user, place, product, video 메타데이터
user = pd.read_csv('../Data/whyout_data/user.csv') # (31177,3)
final_user_interest = pd.read_csv('../Data/whyout_data/final_user_interest.csv') # (31178,15)
place = pd.read_csv('../Data/whyout_data/place.csv') # shape(4697,10)
product = pd.read_csv('../Data/whyout_data/product.csv') # shape(5821,11)
video = pd.read_csv('../Data/whyout_data/video.csv') # shape(3250, 9)

In [7]:
# 유저의 행동데이터
user_place = pd.read_csv('../Data/whyout_data/col_user_place.csv') # (31177,4697)
user_product = pd.read_csv('../Data/whyout_data/col_user_product.csv') # (31177,5821)
user_video = pd.read_csv('../Data/whyout_data/col_user_video.csv') # (31177, 3250)

In [8]:
# 각 아이템 별 행동 데이터가 없는 유저를 삭제한 데이터
drop_user_place = pd.read_csv('../Data/whyout_data/drop_user_place.csv') # (22420,4697) 
drop_user_product = pd.read_csv('../Data/whyout_data/drop_user_product.csv') # (2994,5821)
drop_user_video = pd.read_csv('../Data/whyout_data/drop_user_video.csv') # (11067, 3250)

In [9]:
# 각 아이템 별 행동 데이터가 없는 유저를 삭제한 데이터의 idx
drop_user_place_idx = pd.read_csv('../Data/whyout_data/drop_user_place_idx.csv') # (22420,4)
drop_user_product_idx = pd.read_csv('../Data/whyout_data/drop_user_product_idx.csv') # (2294,4)
drop_user_video_idx = pd.read_csv('../Data/whyout_data/drop_user_video_idx.csv') # (11067, 4)

In [10]:
# 유저의 행동데이터 전체를 사용한 SGD 결과 (U x V)
full_data_sgd_place_preds = pd.read_csv('../Data/whyout_data/sgd_result/full_data/user_place_k50epochs1000.csv')
full_data_sgd_product_preds = pd.read_csv('../Data/whyout_data/sgd_result/full_data/user_product_k50epochs1000.csv')
full_data_sgd_video_preds = pd.read_csv('../Data/whyout_data/sgd_result/full_data/user_video_k50epochs1000.csv')

In [11]:
# 각 아이템 별 행동 데이터가 없는 유저를 삭제한 데이터를 사용한 SGD 결과 (U x V)
del_data_sgd_place_preds = pd.read_csv('../Data/whyout_data/sgd_result/del_data/drop_user_place_k40epochs1000.csv')
del_data_sgd_product_preds = pd.read_csv('../Data/whyout_data/sgd_result/del_data/drop_user_product_k20epochs1000.csv')
del_data_sgd_video_preds = pd.read_csv('../Data/whyout_data/sgd_result/del_data/drop_user_video_k30epochs1000.csv')

In [12]:
# 각 아이템 별 행동 데이터가 없는 유저를 삭제한 데이터를 사용한 SGD 결과의 user_latent(U)와 item_latent(V)
place_user_latent = pd.read_csv('../Data/whyout_data/sgd_result/del_data/drop_user_place_user_latent_k40epochs1000.csv')
product_user_latent = pd.read_csv('../Data/whyout_data/sgd_result/del_data/drop_user_product_user_latent_k20epochs1000.csv')
video_user_latent = pd.read_csv('../Data/whyout_data/sgd_result/del_data/drop_user_video_user_latent_k30epochs1000.csv')

In [79]:
# 신규 유저는 del 하고 관심항목 작성
data = 'del' # all, del
user_id = 37000
new_user_item = [1,0,0]
new_user_outdoor = [0,0,0,0,0,0,0,0,0,1]
num_recommendations = 10
del_dict = { 'place' : [del_data_sgd_place_preds, place, drop_user_place, drop_user_place_idx, place_user_latent, user_place],
         'video' : [del_data_sgd_video_preds, video, drop_user_video, drop_user_video_idx, video_user_latent, user_product],
         'product' : [del_data_sgd_product_preds, product, drop_user_product, drop_user_product_idx, product_user_latent, user_video]}
full_dict = { 'place' : [full_data_sgd_place_preds, place, user_place, user],
         'video' : [full_data_sgd_video_preds, video, user_video, user],
         'product' : [full_data_sgd_product_preds, product, user_product, user]}
item = 'video'
item_list = list(del_dict.keys())

# 모든 데이터 사용
if data == 'all':
    dict = full_dict
    recommend_all(dict[item][0], user_id, dict[item][1], dict[item][2], dict[item][3], num_recommendations)
# 행동 내역을 제거한 데이터 사용
elif data == 'del':
    dict = del_dict
    # if : 기존 유저, else : 신규 유저 구분하기
    if user_id in user['idx'].values:
        recomm_list = recommendation_system(user_id, item, item_list, dict, num_recommendations)
        print(f"user {user_id}에게 추천해줄 {10}개 {item} idx : {recomm_list}")
    else:
        exact_match_indices = find_similar_index(final_user_interest, new_user_item, new_user_outdoor)
        if isinstance(exact_match_indices, (int, np.integer)):
            user_id = int(final_user_interest[final_user_interest.index == exact_match_indices].iloc[:,0])
        else:
            final_match_idx = []
            for i in exact_match_indices:
                i = int(final_user_interest[final_user_interest.index == i].iloc[:,0])
                if i in dict[item][3]['idx'].values:
                    final_match_idx.append(i)
            print(f'{item}에 대한 행동이 있는 user index : {final_match_idx}')
            # 행동이 가장 많은 유저를 선택하는 부분
            if final_match_idx:
                # final_match_idx 중 drop_user_place에서 가장 행동이 많은 유저 선택
                user_actions = dict[item][5].loc[exact_match_indices].apply(lambda row : (row != 0).sum(), axis=1)
                match_user_id = int(final_user_interest[final_user_interest.index == user_actions.idxmax()].iloc[:,0])
                user_actions_index = dict[item][5].loc[user_actions.idxmax()][dict[item][5].loc[user_actions.idxmax()] != 0].columns.tolist()
                # user_actions_index = [int(i) for i in user_actions_index]
                # print(user_actions_index)
                # final_user_actions_index = []
                # #print(int(dict[item][1][dict[item][1].index == 118].iloc[:,0]))
                # for i in user_actions_index:
                #     print(i)
                #     final_user_actions_index.append(int(dict[item][1][dict[item][1].index == i].iloc[:,0]))
                print(f'{item}에 대한 행동이 가장 많은 유저 : user {match_user_id}')

        recom_list2 = recommendation_system(match_user_id, item, item_list, dict, num_recommendations)
        print(f"user {user_id}에게 추천해줄 {num_recommendations}개 {item} idx : {recom_list2}")
else:
    print('데이터를 다시 선택해주세요')

새로운 데이터와 정확히 일치하는 인덱스는 [117, 8724, 10893, 12581, 13681, 16568, 17538, 23082, 23386, 26893, 27744, 27985]입니다.
video에 대한 행동이 있는 user index : [12080, 13853, 14977, 18952, 24726, 25031, 28559, 29413, 29654]
video에 대한 행동이 가장 많은 유저 : user 25031
user 37000에게 추천해줄 10개 video idx : [3477, 1527, 1526, 3539, 1476, 3227, 3328, 3374, 3329, 639]
